# Introduction to Apptainer Containers

**Apptainer** (formerly Singularity) is a container system designed for HPC environments, allowing you to package applications and their dependencies in a portable format.

**Container is available in three states:** 
(i) dockerfile  is instructions to build a container   
(ii) container is a running instance!!!  
(iii) Image    is a file (e.g x.tar or x.sif)  


![Container Overview](../images/container_images.png)


**Container lacks connection to;**  
(i) Your file system  
(ii) Your system ports  
 
You need **internet** connection to build a container from dockerfile but not to run it  



In [2]:
# Display current user, hostname, and where we are
!whoami
!hostname
!pwd


root
ruby047
/scratch/aazd1f17/shared_space/AI_hackathon25/notebooks


In [21]:
ls ../helical/

build_helical_container.sh*  helical_singularity.def*
helical_container.sif*       nohup.out*


In [22]:
cat ../helical/helical_singularity.def

Bootstrap: docker
From : library/python:3.11
#From : nvidia/cuda:12.5.1-cudnn-devel-ubuntu22.04

%labels
    Author maxime@helical-ai.com
    Author benoit@helical-ai.com
    Version v0.0.1

%post
   cd /home
   apt-get -y upgrade 
   apt-get -y update
   apt-get install -y cmake wget git curl gcc gfortran 
   apt-get clean
   python3 -m pip install helical


In [18]:
# Root file system contents
!ls /
!ls /bin | grep Onto


bin   environment  lib	  mnt	root  scratch	   sys	var
boot  etc	   lib64  opt	run   singularity  tmp	workspace
dev   home	   media  proc	sbin  srv	   usr


In [24]:
!cat ../run_jupyter.sh

# This script launches a Jupyter Notebook server inside an Apptainer container.
# It sets the server to run on port 8888 and binds the current directory to the container.

# Load the Apptainer module (required to use apptainer commands)
module load apptainer

# Run the Apptainer container using `exec` to execute a command inside it
apptainer exec \

  # Set the home directory inside the container to the current directory
  -H $PWD \

  # Bind the current working directory to /workspace inside the container
  --bind "$PWD:/workspace" \

  # Enable GPU access (if available) and allow the container to act as root
  --nv --fakeroot \

  # Specify the container image to use
  helical/helical_container.sif \

  # Start the Jupyter Notebook server inside the container
  .local/bin/jupyter notebook \
    --ip=0.0.0.0 \            # Allow access from any IP address
    --port=8888 \             # Set the port to 8888
    --no-browser \            # Do not open a browser automatically
    --allo